In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util, CrossEncoder, InputExample, losses, models, datasets
import torch
import os
import csv
import pickle
import glob
from pprint import pprint
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
from torch import nn
import random
import gc
import time

In [2]:
data = pd.read_csv('filename.csv')
# data=data.dropna(subset=['Skills','LanguageSkills']).reset_index(drop=True)
data['Skills']=data['Skills'].apply(lambda x: x.replace('\n',' ')[9:].strip())
data['LanguageSkills']=data['LanguageSkills'].apply(lambda x: x.replace('\n',' ')[12:].strip())

In [3]:
data.head()

,Name,Location,Email,Gender,Date Of Birth,Marital Status,Phone Number,Country,Summary,Objective,Skills,SoftSkills,LanguageSkills,Projects,Certifications/Awards,Education,Experience,Activities,Hobbies/Interests,path
0,Bui Quoc Khanh,Ho Chi Minh City,bkdev98@gmail.com,NaN,NaN,NaN,1.120170e+11,V,NaN,NaN,OURCE CODE CUSTOMER RELATIONSHIP MANAGEMENT AP...,DESIGN,,React Native Boilerplate 2017 https://github.c...,NaN,NaN,Front-end Web Developer Thich Lam Moc 03/2017 ...,NaN,NaN,C:\Users\tuan.ton\Desktop\data_resume_for_sear...
1,Bui Phan Tho (Eric),Ho Chi Minh,buiphantho@gmail.com,NaN,1999-01-06,NaN,3.490049e+08,V,"I'm a software engineer, love technologies, pr...",NaN,ES RABBITMQ API .NET WEB APP REACT NATIVE SOFT...,CHOOSE COMMUNICATION TEAMWORK ORGANIZATION AGI...,,NaN,NaN,November 2019 - September 2020 Fresher .NET en...,November 2019 - August 2020 CRM and LMS produc...,2018 - 2021 Company tours for students I had a...,"INTERESTS Teaching programming, reading, trave...",C:\Users\tuan.ton\Desktop\data_resume_for_sear...
2,Tran Cong Tien Hung,"391 Cong Hoa Street, Tan Binh District",tienhung0182@gmail.com,Male,1997-10-28,NaN,2.819971e+11,NaN,NaN,Learn as much as possible about the life cycle...,TE DOCKER AZURE GRPC KAFKA ANGULAR SERVER SOFT...,COMMUNICATION,,April 2020 >> Oct 2020 EDT AGITATOR MICROSERVI...,CERTIFICATIONS 2016 7.5 IELTS CERTIFICATE BY B...,NaN,April 2020 >> Current FPT SOFTWARE - HALLIBURT...,NaN,INTERESTS Video Games Working out Philosophy a...,C:\Users\tuan.ton\Desktop\data_resume_for_sear...
3,Dang Chien Nguyen,DANG CHIEN NGUYEN Hanoi,dangchienhsgs@gmail.com,NaN,NaN,NaN,8.438350e+10,V,NaN,NaN,EM RABBITMQ AIRFLOW HADOOP ML PANDAS HBASE DAT...,ACCURACY CHOOSE CRITICAL THINKING COMMUNICATIO...,,NaN,CERTIFICATIONS Honor title Second Prize of Awa...,"HIGH SCHOOL FOR GIFTED STUDENTS, VIETNAM NATIO...",Data Science Laboratory (Hanoi University of S...,NaN,NaN,C:\Users\tuan.ton\Desktop\data_resume_for_sear...
4,Personal Detail Objective Experience & Skill...,"517/14 Phan Van Tri Street, Ward 5, Go Vap Dis...",dqduc.1512121@gmail.com,Male,1997-02-01,NaN,8.433828e+10,NaN,NaN,Desired position: Fullstack Developer Career ...,FLUTTER SOFTWARE ASP.NET GITLAB CRYSTAL GITHUB...,CONFIDENT TEAM WORKING CREATIVITY DESIGN RESPO...,,4.Self-learning: - Backend: .Net Core 2.2 - F...,NaN,2015-2020 University of Science HCMC Major: ...,- 2 years experience in C# development on Wind...,NaN,NaN,C:\Users\tuan.ton\Desktop\data_resume_for_sear...


In [14]:
#We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
bi_encoder = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')
top_k = 10     #Number of passages we want to retrieve with the bi-encoder

#The bi-encoder will retrieve k documents. We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2',max_length=512)

passages=data['embeddedString'].values.tolist()
passages_vector = bi_encoder.encode(passages)

#If you like, you can also limit the number of passages you want to use
print("Length of passages: ", len(passages))


def search(query, passages_vector, passages, top_k):
    print("Input question:", query)

    ##### Sematic Search #####
    # Encode the query using the bi_encoder and find potentially relevant passages
    t=time.time()
    query_vector = bi_encoder.encode([query])
    cosine_similarities = util.dot_score(query_vector, passages_vector)
    cosine_similarities_sorted, index = torch.sort(cosine_similarities)
    index = index[0][:top_k]
    print('>>>> Results in Total Time when we run bi-encoder: {}'.format(time.time()-t))

    ##### Re-Ranking #####
    # Score all the retrieved passages with the cross_encoder
    cross_encoder_list = []
    for i in index.tolist():
        cross_encoder_list.append((query, passages[i]))
    scores = cross_encoder.predict(cross_encoder_list)


    return scores, index
    
    


Length of passages:  50


In [15]:
score, index = search('', passages_vector, passages, top_k)

Input question: python
>>>> Results in Total Time when we run bi-encoder: 0.038001060485839844


In [16]:
print(score)
index = index.tolist()
print(index)
d = dict(zip(index, score))
print(d)
output = {k: v for k, v in sorted(d.items(), key=lambda item: item[1])}
print(output)

[-11.356279  -11.356279  -11.327574  -11.50591   -11.285542  -11.3566475
 -11.350462  -11.4878845 -11.420981  -11.261401 ]
[34, 29, 40, 15, 38, 41, 20, 12, 42, 9]
{34: -11.356279, 29: -11.356279, 40: -11.327574, 15: -11.50591, 38: -11.285542, 41: -11.3566475, 20: -11.350462, 12: -11.4878845, 42: -11.420981, 9: -11.261401}
{15: -11.50591, 12: -11.4878845, 42: -11.420981, 41: -11.3566475, 34: -11.356279, 29: -11.356279, 20: -11.350462, 40: -11.327574, 38: -11.285542, 9: -11.261401}


In [7]:
# sort_list = sorted(score, key = lambda x:float(x))
# print(sort_list)
# sort_index = []
# for i in range(top_k):
#     for j in sort_list:
#         if d[i].values() == j:
#             sort_index.append(d[i].keys())
# print(sort_index)

In [9]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2-v2', max_length=512)
scores = model.predict([('Query1', 'Paragraph1'), ('Query1', 'Paragraph2')])

#For Example
scores = model.predict([('How many people live in Berlin?', 'Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.'), 
                        ('How many people live in Berlin?', 'Berlin is well known for its museums.')])

In [10]:
print(scores)

[ 7.1523695 -6.2870455]
